In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [89]:
#Configuration

K = 5 #Number of mixture models
eps = 1e-10
in_video_file_path = 'Data/umcp.mpg' #path to input audio
out_path = '' #directory for foreground and background videos

In [3]:
in_video = cv2.VideoCapture(in_video_file_path)

In [4]:

K = 5

In [5]:
in_video.read()[1].shape

(240, 352, 3)

In [60]:
#model

#frame_size

frame_size = (in_video.read()[1]).shape[:2]
frame_size = (2,2)
#For each pixel, K set of mean,var and pi are required; Where mean and variance can be vectors and pi is scalar
pi = np.ones((*frame_size,K))/K # pi-> (MxNxK)
mean = np.zeros((*frame_size,3,K))
var = np.ones((*frame_size,3,K))/K/K #Covariance matrix is assumed to be diagonal
x = np.random.randn(*frame_size,3,1)

In [27]:
(mean-x).shape, var.shape

((2, 3, 3, 5), (2, 3, 3, 5))

In [29]:
((mean-x)* var).shape

(2, 3, 3, 5)

In [46]:
a = np.arange(2*2*3*5).reshape(2,2,3,5)
a_swap = np.swapaxes(a,2,3)
print(a[0,0,:,4],a_swap[0,0,4,:])

b = np.ones(2*2*3*5).reshape(2,2,3,5)
b

[ 4  9 14] [ 4  9 14]


array([[[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]]])

In [37]:
a = np.array([2,2,3])
b = np.array([[4],[1],[2]])
np.matmul(b,a)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)

In [97]:
# utility functions

def gpdf(x,mean,var):
    #gaussian probability density function
    mahalD = np.sum(((x-mean)* (1/(var+eps)))*(x-mean),axis=2)
    return 1/(np.sqrt(2*np.pi*np.sum(var*var,axis=2)))

In [96]:
np.pi

3.141592653589793

In [98]:
#x = np.ones_like(mean)

gpdf(x,mean,var).shape

(2, 2, 5)

In [67]:
x[:,:,:,0]-mean[:,:,:,0],var[:,:,:,0]

(array([[[ 0.65854413, -0.54424687, -0.4596294 ],
         [-1.77696123, -0.55935803, -0.09846363]],
 
        [[ 0.59684634,  1.14784853, -0.53364428],
         [ 0.08879543,  0.77681138, -0.86317912]]]),
 array([[[0.04, 0.04, 0.04],
         [0.04, 0.04, 0.04]],
 
        [[0.04, 0.04, 0.04],
         [0.04, 0.04, 0.04]]]))

In [75]:
x[:,:,:,0]

array([[[ 0.65854413, -0.54424687, -0.4596294 ],
        [-1.77696123, -0.55935803, -0.09846363]],

       [[ 0.59684634,  1.14784853, -0.53364428],
        [ 0.08879543,  0.77681138, -0.86317912]]])

In [76]:
np.sum((x-mean)* var*(x-mean),axis=2)[:,:,0]

array([[0.03764577, 0.13920671],
       [0.07834232, 0.05425595]])

In [85]:
(0.65854413**2)*0.04 + (0.54424687**2)*0.04 + 0.4596294**2*0.04

0.03764576848026456

In [84]:
(0.65854413**2)*0.04,(-0.54424687**2)*0.04

(0.017347214846298278, -0.011848186220191879)

In [83]:
((x-mean)* var*(x-mean))[:,:,:,0]

array([[[0.01734721, 0.01184819, 0.00845037],
        [0.12630365, 0.01251526, 0.0003878 ]],

       [[0.01424902, 0.05270225, 0.01139105],
        [0.00031539, 0.02413744, 0.02980313]]])